# Tutorial: nc-check Plugin Architecture Demo

Audience:
- Developers and researchers validating geospatial `xarray` datasets.

Prerequisites:
- Basic Python and `xarray` familiarity.
- This repository checked out locally.

Learning goals:
- Canonicalize datasets to `time`, `lat`, `lon`.
- Write and run atomic checks.
- Compose checks into a `CheckSuite`.
- Use plugin registration and python-first reporting.


## 1) Setup

In [ ]:
from __future__ import annotations

from pathlib import Path

import numpy as np
import xarray as xr

import nc_check
from nc_check.models import AtomicCheckResult
from nc_check.suite import CheckDefinition

## 2) Canonicalize to `time`, `lat`, `lon`

In [2]:
raw = xr.Dataset(
    data_vars={
        "temp": (
            ("t", "latitude", "longitude"),
            np.array([[[280.0, 281.2], [279.8, 280.1]]]),
        )
    },
    coords={
        "t": ("t", [0], {"units": "days since 2000-01-01 00:00:00"}),
        "latitude": ("latitude", [-10.0, 10.0], {"units": "degrees_north"}),
        "longitude": ("longitude", [0.0, 180.0], {"units": "degrees_east"}),
    },
    attrs={"Conventions": "CF-1.12"},
)

canonical = nc_check.canonicalize_dataset(raw)
{
    "data_dims": canonical["temp"].dims,
    "coords": list(canonical.coords),
    "coord_names": canonical.coordinate_names,
}

{'data_dims': ('time', 'lat', 'lon'),
 'coords': ['time', 'lat', 'lon'],
 'coord_names': CoordinateNames(time='time', lat='lat', lon='lon')}

## 3) Atomic checks and a custom suite

In [4]:
def check_temp_has_values(ds):
    if np.isnan(ds["temp"].values).all():
        return AtomicCheckResult.failed_result(
            name="demo.temp_has_values",
            info="All temp values are NaN.",
        )
    return AtomicCheckResult.passed_result(
        name="demo.temp_has_values",
        info="Temp has at least one non-NaN value.",
    )


def check_time_coord_present(ds):
    if "time" not in ds.coords:
        return AtomicCheckResult.failed_result(
            name="demo.time_coord_present",
            info="time coordinate is missing.",
        )
    return AtomicCheckResult.passed_result(
        name="demo.time_coord_present",
        info="time coordinate is present.",
    )


single_result = nc_check.run_atomic_check(
    canonical,
    CheckDefinition(name="demo.temp_has_values", check=check_temp_has_values),
)

suite = nc_check.CheckSuite(
    name="demo_suite",
    checks=[
        CheckDefinition(name="demo.temp_has_values", check=check_temp_has_values),
        CheckDefinition(name="demo.time_coord_present", check=check_time_coord_present),
    ],
)
suite_report = suite.run(canonical)

{
    "single_check": single_result.to_dict(),
    "suite_report": suite_report.to_dict(),
}

{'single_check': {'name': 'demo.temp_has_values',
  'status': 'passed',
  'info': 'Temp has at least one non-NaN value.',
  'details': {}},
 'suite_report': {'suite_name': 'demo_suite',
  'plugin': None,
  'checks': [{'name': 'demo.temp_has_values',
    'status': 'passed',
    'info': 'Temp has at least one non-NaN value.',
    'details': {}},
   {'name': 'demo.time_coord_present',
    'status': 'passed',
    'info': 'time coordinate is present.',
    'details': {}}],
  'summary': {'checks_run': 2,
   'passed': 2,
   'skipped': 0,
   'failed': 0,
   'overall_status': 'passed'}}}

## 4) Built-in CF compliance plugin suite

In [5]:
cf_report = nc_check.run_cf_compliance(canonical)
cf_payload = cf_report.to_dict()

{
    "summary": cf_payload["summary"],
    "statuses": {item["name"]: item["status"] for item in cf_payload["checks"]},
}

{'summary': {'checks_run': 6,
  'passed': 6,
  'skipped': 0,
  'failed': 0,
  'overall_status': 'passed'},
 'statuses': {'cf.conventions': 'passed',
  'cf.coordinates_present': 'passed',
  'cf.latitude_units': 'passed',
  'cf.longitude_units': 'passed',
  'cf.time_units': 'passed',
  'cf.coordinate_ranges': 'passed'}}

## 5) Python-first report -> HTML

In [6]:
html_text = nc_check.render_html_report(cf_report)
html_path = Path("output/jupyter-notebook/cf-compliance-demo-report.html")
nc_check.save_html_report(cf_report, html_path)

{
    "saved_html": str(html_path),
    "html_length": len(html_text),
    "preview": html_text[:180],
}

{'saved_html': 'output/jupyter-notebook/cf-compliance-demo-report.html',
 'html_length': 1762,
 'preview': "<!doctype html><html><head><meta charset='utf-8'><title>nc-check report</title><style>body{font-family:ui-sans-serif,system-ui,-apple-system,Segoe UI,sans-serif;margin:2rem;color:#"}

## 6) Register a custom plugin and combine checks

In [8]:
class NoNegativeLatPlugin:
    name = "custom_lat"

    def register(self, registry):
        def check_non_negative_lat(ds):
            min_lat = float(ds.coords["lat"].min())
            if min_lat < 0:
                return AtomicCheckResult.failed_result(
                    name="custom.non_negative_lat",
                    info="Latitude contains negative values.",
                    details={"min_lat": min_lat},
                )
            return AtomicCheckResult.passed_result(
                name="custom.non_negative_lat",
                info="All latitude values are non-negative.",
                details={"min_lat": min_lat},
            )

        registry.register_check(
            name="custom.non_negative_lat",
            check=check_non_negative_lat,
            plugin=self.name,
        )


registry = nc_check.create_registry(load_entrypoints=False)
registry.register_plugin(NoNegativeLatPlugin())

combined_report = nc_check.run_suite(
    canonical,
    suite_name="combined_suite",
    check_names=["cf.conventions", "custom.non_negative_lat"],
    registry=registry,
)
combined_report.to_dict()

{'suite_name': 'combined_suite',
 'plugin': None,
 'checks': [{'name': 'cf.conventions',
   'status': 'passed',
   'info': 'Conventions includes a CF token.',
   'details': {'conventions': 'CF-1.12'}},
  {'name': 'custom.non_negative_lat',
   'status': 'failed',
   'info': 'Latitude contains negative values.',
   'details': {'min_lat': -10.0}}],
 'summary': {'checks_run': 2,
  'passed': 1,
  'skipped': 0,
  'failed': 1,
  'overall_status': 'failed'}}

## Next steps

- Add your own plugin package and expose checks via `nc_check.plugins` entry points.
- Build domain-specific suites by selecting check names and reusing one registry.
- Keep checks atomic so each failure is easy to diagnose.
